### In this notebook I am using pytorch as it is really suitable for building from scratch neural networks
### I will build a variational autoencoder for 1D input data specifically time-series 

In [2]:
import os, time
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [3]:
depth = 16      # initial depth to convolve channels into
filt_size = 4   # convolution filter size
stride = 2      # stride for conv
pad = 1         # padding added for conv

In [ ]:
# Encoder 

In [ ]:
        self.encoder = nn.Sequential()  
        # input: n_channels x size
        # ouput: depth x conv_size
        # conv_size = (size - filt_size + 2 * pad) / stride + 1
        self.encoder.add_module('input1', nn.Conv1d(n_channels, depth,
                                                        filt_size, stride, pad,
                                                        bias=True))
        self.encoder.add_module('input2', nn.ReLU(inplace=True))
        
        # Add conv layer 
        # Pyramid strategy pooling and batch normalization 
        for i in range(n - 3):
            # input: i_depth x conv_size
            # output: o_depth x conv_size
            # i_depth = o_depth of previous layer
            i_depth = depth * 2 ** i
            o_depth = depth * 2 ** (i + 1)
            self.encoder.add_module(f'pyramid_{i_depth}-{o_depth}_conv',
                                    nn.Conv1d(i_depth, o_depth, filt_size, stride, pad, bias=True))
            self.encoder.add_module(f'pyramid_{o_depth}_batchnorm',
                                    nn.BatchNorm1d(o_depth))
            self.encoder.add_module(f'pyramid_{o_depth}_relu',
                                    nn.ReLU(inplace=True))

In [ ]:
# Latent space

In [ ]:
# Convolution of encoded vector into the latent space
max_depth = depth * 2 ** (n - 3)
self.conv_mu = nn.Conv1d(max_depth, n_latent, filt_size)
self.conv_logvar = nn.Conv1d(max_depth, n_latent, filt_size)

In [ ]:
# Decoder - second half of VAE

In [ ]:
self.decoder = nn.Sequential()
# input: max_depth x conv_size
# output: n_latent x conv_size
# default stride=1, pad=0 for this layer
self.decoder.add_module('input1', nn.ConvTranspose1d(n_latent, max_depth, filt_size, bias=True))
self.decoder.add_module('input2', nn.BatchNorm1d(max_depth))
self.decoder.add_module('input3', nn.ReLU(inplace=True))
    
# Reverse the convolution pyramids used in the encoder
for i in range(n - 3, 0, -1):
    
    i_depth = depth * 2 ** i
    o_depth = depth * 2 ** (i - 1)
    self.decoder.add_module(f'pyramid_{i_depth}-{o_depth}_conv',
                                    nn.ConvTranspose1d(i_depth, o_depth, filt_size, stride, pad, bias=True))
    self.decoder.add_module(f'pyramid_{o_depth}_batchnorm',
                                    nn.BatchNorm1d(o_depth))
    self.decoder.add_module(f'pyramid_{o_depth}_relu', nn.ReLU(inplace=True))
        
# Final transposed convolution to return to vector size
# TODO: ?No final activation to allow unbounded numerical output
    self.decoder.add_module('output-conv', nn.ConvTranspose1d(depth, n_channels,
                                                                  filt_size, stride, pad,
                                                                  bias=True))

In [ ]:
### Loss function

In [ ]:
# Reconstruction loss
        batch_size = trans.shape[0]
    
        gen_err = (trans - gen_trans).pow(2).reshape(batch_size, -1)
        gen_err = 0.5 * torch.sum(gen_err, dim=-1)  
        
        if reduce:
            gen_err = torch.mean(gen_err)
        
        # Regularizer

        KL = (-logvar + logvar.exp() + mu.pow(2) - 1) * 0.5
        KL = torch.sum(KL, dim=-1)
        
        if reduce:
            KL = torch.mean(KL)
            
        loss = gen_err + self.beta * KL